
# MLflow Tracking Demo – Proyecto MLOps Fase 2

Este notebook muestra cómo ejecutar el **pipeline** de entrenamiento y evaluación del proyecto, 
registrando **parámetros, métricas y artefactos** en **MLflow**.  
Sirve como evidencia para los criterios de *métodos y técnicas*, *resultados* y *reproducibilidad*.



## 1) Requisitos previos

Antes de ejecutar:
- Instala dependencias: `pip install -r requirements.txt`
- Verifica que el dataset crudo esté en `data/raw/`
- (Opcional) Tener DVC inicializado: `dvc init`


In [ ]:

# Configuración e imports
from pathlib import Path
import mlflow, mlflow.sklearn
import joblib
import matplotlib.pyplot as plt

# Módulos del proyecto (ajustados a la estructura Fase 2)
from fase2.config import Settings
from fase2.modeling import train as train_mod
from fase2.modeling import predict as predict_mod

# MLflow local (carpeta mlruns/ en el repo)
mlflow.set_tracking_uri("mlruns")
mlflow.set_experiment("german_credit_exps")

cfg = Settings()
print("Tracking URI:", mlflow.get_tracking_uri())
print("Experiment:", "german_credit_exps")



## 2) (Opcional) Preparación y *feature engineering*

Si no has generado los conjuntos `data/processed/`, ejecuta en terminal antes de correr las celdas:
```bash
python -m fase2.dataset
python -m fase2.features
```


## 3) Entrenamiento con tracking en MLflow

In [ ]:

result = train_mod.train(model_name="random_forest")
print(">> Best CV AUC:", result.get("best_score"))
print(">> Best Params:", result.get("best_params"))


## 4) Evaluación en test + registro de métricas

In [ ]:

metrics = predict_mod.evaluate(model_name="random_forest")
metrics


## 5) Visualizaciones locales (matriz de confusión y curva ROC)

In [ ]:

from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

X_test = cfg.load_dataframe(Path(cfg.paths.processed) / "X_test.csv")
y_test = cfg.load_series(Path(cfg.paths.processed) / "y_test.csv")

model_path = Path(cfg.paths.models) / "random_forest_pipeline.pkl"
model = joblib.load(model_path)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Confusion Matrix
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title("Confusion Matrix – Random Forest")
plt.show()

# ROC Curve
RocCurveDisplay.from_predictions(y_test, y_prob)
plt.title("ROC Curve – Random Forest")
plt.show()


## 6) Explorar *runs* en MLflow (desde Python)

In [ ]:

# Nota: para UI web, ejecuta en terminal: mlflow ui
client = mlflow.tracking.MlflowClient()
exp = client.get_experiment_by_name("german_credit_exps")
runs = client.search_runs([exp.experiment_id], order_by=["attributes.start_time DESC"], max_results=5)
[(r.info.run_id, r.data.metrics) for r in runs]



## 7) Conclusiones técnicas

- El pipeline se ejecuta end-to-end y registra **parámetros, métricas y artefactos** en MLflow.  
- Las figuras se guardan también en `reports/figures/` y se suben como artefactos.  
- Esta evidencia respalda los criterios de **métodos y técnicas**, **resultados** y **gestión profesional** del proyecto.
